In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library.standard_gates import RYGate
from qiskit.circuit.library import StatePreparation, XGate, MCXGate

import numpy as np
from bigtree import BinaryNode, print_tree, tree_to_dict, levelorder_iter, preorder_iter

def decimal_to_binary_fixed_length(number, length):
    binary_representation = format(number, 'b')  # Convert to binary without '0b' prefix
    return binary_representation.zfill(length)  # Pad with zeros to the left to get the fixed length
    
def test_bool(tupel):
    return (tupel[0] and tupel[1]) or (tupel[0] and tupel[2]) or  (tupel[1] and tupel[2])

def test_13(tupel):
    tupel_temp = tupel.copy()
    tupel_temp.reverse()
    binary_string = ''.join(str(bit) for bit in tupel_temp)
    integer_value = int(binary_string, 2)
    #print("Int=", integer_value)
    if integer_value <= 43:
        #print(integer_value, 1)
        return 1
    else:
        #print(integer_value, 0)
        return 0
    
def Spieler4Karten2Stiche(tupel):
    tupel_temp = tupel.copy()
    cardquantity= 4 
    cardbits = 3
    cards=[]
    ctr1=0
    ctr2=0
    for i in range(cardquantity):
        cards+=[tupel_temp[cardbits*i: cardbits*(i+1)]]
        if cards[i]==[0,0,0]:
            ctr1+=1
        elif cards[i]==[0,0,1]:
            ctr2+=1

    if ctr1 != 2: 
        return 0

    if ctr2 != 2: 
        return 0

    return 1

def test_one(tupel):
    return 1

def test_or(tupel):
    return (tupel[0] or tupel[1])

def test_and(tupel):
    return (tupel[0] and tupel[1])

def valid_cards2(cards):
    # card|player-state with 2 players and 4 cards, 1 round
    c = 2 #number of card-bits
    s = 3 #number of playerstate-bits
    p = 2 #number of players
    n = (c+s) #length of card-tupel
    
    pc_dict = {}
    #print(players)
    #print(cards)

    for i in range(2):
        player =  cards[i*n+c]
        card = 2**0*cards[i*n+1]+2**1*cards[i*n+0]
        # print('card', card, 'belongs to player', player)
        
        if player in pc_dict.keys():
            # print("Spieler kann nicht mehrere Karten haben\n")
            return 0
        else:
            if card in pc_dict.values():
                # print("Karte kann nur zu einem Spieler gehören\n")
                return 0
            else:
                pc_dict.update({player:card})
                
    # print("Verteilung", cards," ist zulässsig\n")
    return 1

def get_projectionprobs(f, n, projection_bit):
    #f .... boolean function on n bits
    p_proj = 0
    p_full = 0
    for i in range(2**n):
        argument = []
        #for k in range(n):
        #    argument+= [i//2**(n-1-k)] 
        #    i = i % 2**(n-1-k)
        binary_str = decimal_to_binary_fixed_length(i, n)
        argument = [int(digit) for digit in binary_str]

               
        if argument[projection_bit] == 0:
            p_proj = p_proj + f(argument)
        p_full = p_full + f(argument)
        #print(argument, f(argument) , p_proj, p_full)
    #Define restricted functions
    def f_one(tupel):
        tupel_temp = tupel.copy()
        tupel_temp.insert(projection_bit,1)
        return f(tupel_temp)
    def f_zero(tupel):
        tupel_temp = tupel.copy()
        tupel_temp.insert(projection_bit,0)
        return f(tupel_temp)
    
    if p_full > 0:
        p = p_proj/p_full
    else:
        p = -1
    
    return p, f_one, f_zero
    



def get_fulltree(f,n,prefix=""):
    
    p, f_left, f_right = get_projectionprobs(f, n, 0)
    if p>= 0:
        root = BinaryNode(prefix, p = p)
        #print(prefix,"\t n=",n,"p=",p)
    
    else:
        return
    
    if n > 1:
        root.children = [get_fulltree(f_left,n-1,prefix+"1"),get_fulltree(f_right,n-1,prefix+"0")]
    #if n == 1:
    #    print(f([0]), f([1]))   
    return root


def p_gate(p,circuit,qubit):
    theta = 2*np.arccos(np.sqrt(p))
    circuit.ry(theta,qubit)

    
  
    
def make_qsp(f,n,circuit):
    gates = get_fulltree(f,n) 
    #gates.show(attr_list=['p'])
    for d in range(1,n+1): # root extra
        #for node in levelorder_iter(gates, filter_condition = lambda x: x.depth == d):
        for node in preorder_iter(gates, filter_condition = lambda x: x.depth == d):
            #cond = node.node_name # noch ein string
            cond = ''.join(reversed(node.node_name))
            #for i in range(len(cond)):
            #    cond[i] = int(cond[i])
            p = node.get_attr("p")
            work_qubit = node.depth-1
            cond_qubits = range(0,work_qubit)
            #print(work_qubit, cond_qubits, cond,p)
            
            theta = 2*np.arccos(np.sqrt(p))
            if d > 1:
                p_gate = RYGate(theta).control(len(cond), ctrl_state=cond )
            else:
                p_gate = RYGate(theta)
            circuit.append(p_gate, range(node.depth))

        
#print("two-out-of-three:" )       
#root = get_fulltree(test_bool,3)
#print_tree(root,attr_list=["p"])
#tree_to_dict(root,name_key="node")
#print("\n and:")
#get_fulltree(test_and,2).hshow()
#print("\n or:")
#print_tree(get_fulltree(test_or,2),attr_list=["p"])
#print("\n constant one:")
#get_fulltree(test_one,2).hshow()

N= 12
qr = QuantumRegister(N+5) #5 ancilla qubits
qc = QuantumCircuit(qr)  

make_qsp(Spieler4Karten2Stiche,N, qc)
#qc.measure_all()
#qc.draw()

from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer.primitives import Sampler
 
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram, plot_state_paulivec
from qiskit.primitives import BackendSampler


#quasi_dists = Sampler().run(qc, shots=10000).result().quasi_dists[0].binary_probabilities()
#print(quasi_dists)

#plot_histogram(quasi_dists)


ModuleNotFoundError: No module named 'qiskit'

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
N= 2
n=0
m=1
qr1 = QuantumRegister(N+1)
qcausspiel = QuantumCircuit(qr1)  
qcausspiel.h(N)
qcausspiel.cx(N,n)
qcausspiel.x(N)
qcausspiel.cx(N,m)

#Qubit N wieder auf Null setzen:
qcausspiel.cx(m,N)

#Ausspielen einer von zwei möglichen Karten
ausspiel = qcausspiel.to_gate(label='Ausspiel')

In [ ]:
#Erzeugen eines Gates zum auf Stapel legen
M=4
qr2 = QuantumRegister(M)
qcstapel = QuantumCircuit(qr2) 
qcstapel.x(0)
qcstapel.x(2)
qcstapel.x(3)
stapel = qcstapel.to_gate()

In [ ]:
from datetime import datetime

current_time = datetime.now()

conds=['1100','1010','1001','0101','0110','0011']

#p_gate = RYGate(0).control(11, ctrl_state=cond )
#Superposition aller möglichen Ausspiele Spieler 1
for cond in conds:
    a_gate = ausspiel.control(num_ctrl_qubits= len(cond), ctrl_state=cond, label = 'Ausspiel1')
    pos = []
    for i in range(len(cond)):
        if cond[i] == '1': 
            pos.append(3*i+1)
    qc.append(a_gate,[2,5,8,11,pos[0],pos[1],12])


#Superposition aller möglichen Ausspiele Spieler 2
for cond in conds:
    a_gate = ausspiel.control(num_ctrl_qubits= len(cond), ctrl_state=cond, label = 'Ausspiel2')
    pos = []
    for i in range(len(cond)):
        if cond[i] == '0': 
            pos.append(3*i+1)
    qc.append(a_gate,[2,5,8,11,pos[0],pos[1],12])


condstisch=['1100','1010','1001','0101','0110','0011']
#condstisch=['0011']
#condstisch=['0011','1001']
#Auf Stapel des Spielers legen, der die höchste Karte gespielt hat

for cond in condstisch:
    #t_gate = stapel.control(num_ctrl_qubits= len(cond), ctrl_state=cond)

    #Bedingung in Qiskit-Notation erzeugen
    cstate= cond[3]+'0'+cond[2]+'0'+cond[1]+'0'+cond[0]+'0'
    #print(cstate)

    t_gate = stapel.control(num_ctrl_qubits= 2* len(cond), ctrl_state=cstate, label='Stapel')
    #r_gate = rueck.control(num_ctrl_qubits= 2* len(cond), ctrl_state=cstate, label='Rueck')

    pos = []
    for i in range(len(cond)):
        if cond[i] == '1': 
            for j in range(3):
                pos.append(3*i+j)

    #Ancilla bits füllen
    qc.cx(0,13)
    qc.cx(3,14)
    qc.cx(6,15)
    qc.cx(9,16)
    #qc.append(t_gate,[1,4,7,10,pos[0],pos[2],pos[3],pos[5]])
    #qc.append(t_gate,[10,9,7,6,4,3,1,0,pos[0],pos[2],pos[3],pos[5]])
    #qc.append(t_gate,[10,16,7,15,4,14,1,13,pos[0],pos[2],pos[3],pos[5]])
    qc.append(t_gate,[13,1,14,4,15,7,16,10,pos[0],pos[2],pos[3],pos[5]])
    
    #print(pos[0],pos[2],pos[3],pos[5])
    #print(13+pos[0]//3,13+pos[3]//3 )
    
    #Ancilla bits wieder zurücksetzen
    cxbits=[]
    resetbits=[]
    #print(pos[0], pos[3])
    #for ancillas in [0,3,6,9]:
    #    if (ancillas == pos[0]) or (ancillas == pos[3]):
    #       resetbits.append(ancillas)
    #    else:
    #        qc.cx(ancillas, 13+ancillas//3)
    #        print("ancilla= ", ancillas)
    resetbits=[13,14,15,16]
    qc.reset(resetbits)

#Ausspielen der beiden letzten Karten    
qc.cx(1,12)
qc.cx(4,13)
qc.cx(7,14)
qc.cx(10,15)
#l_gate = MCXGate(num_ctrl_qubits= 2, ctrl_state='00')

for card in range(4):
    #qc.append(l_gate,[3*card,12+card,3*card+1])
    qc.x(3*card)
    qc.x(12+card)
    qc.mcx([3*card,12+card], 3*card+1)
    qc.x(3*card)
    qc.x(12+card)
    #print(3*card,12+card,3*card+1)
resetbits=[12,13,14,15]
qc.reset(resetbits)
#qc.draw()

condstisch=['1100','1010','1001','0101','0110','0011']
#condstisch=['0110']
#condstisch=['0110','1001']
#Auf Stapel des Spielers legen, der die höchste Karte gespielt hat

for cond in condstisch:
    #t_gate = stapel.control(num_ctrl_qubits= len(cond), ctrl_state=cond)

    #Bedingung in Qiskit-Notation erzeugen
    cstate= cond[3]+cond[2]+cond[1]+cond[0]
    print(cond)

    l_gate = stapel.control(num_ctrl_qubits= len(cond), ctrl_state=cstate, label='Stapel')


    pos = []
    for i in range(len(cond)):
        if cond[i] == '0': 
            for j in range(3):
                pos.append(3*i+j)

    #Ancilla bits füllen
    qc.cx(0,13)
    qc.cx(3,14)
    qc.cx(6,15)
    qc.cx(9,16)

    qc.append(l_gate,[13,14,15,16,pos[0],pos[2],pos[3],pos[5]])
    
    print(pos[0],pos[2],pos[3],pos[5])
    #print(13+pos[0]//3,13+pos[3]//3 )
    
    #Ancilla bits wieder zurücksetzen
    resetbits=[13,14,15,16]
    qc.reset(resetbits)

qc.measure_all()


print("Sampling: ", datetime.now())
quasi_dists_neu = Sampler().run(qc, shots=1000).result().quasi_dists[0].binary_probabilities()

print("Histogram: ", datetime.now())
histo = plot_histogram(quasi_dists_neu)
histo.savefig('histogram.jpg')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bitstrings = list(quasi_dists_neu.keys())
frequencies = list(quasi_dists_neu.values())

plt.title('Verteilung nach Spielende')
plt.xlabel('Karten auf Stapel von Spieler 1')
plt.ylabel('Wahrscheinlichkeit')

custom_labels = [
    'K1+4', 'K2+3', 'K1+2', 'alle', 
    'K1+3', 'K2+4', 'keine', 'K3+4'
]

plt.bar(bitstrings, frequencies)
plt.xticks(ticks=range(len(bitstrings)), labels = custom_labels, rotation=90)
plt.savefig('Stapelverteilung.png')

plt.show() 

In [ ]:
####################################################
cards = 4
c_vals = [10,5,1,1]

########## expectation player 1
expect = 0
for state in bitstrings:
    pl_string = state[16:4:-1]
    pl_string = [int(pl_string[i]) for i in [2,5,8,11]]
    # print(pl_string)
    s_expect = 0 
    for i in range(len(c_vals)):
        s_expect += (1-pl_string[i])*c_vals[i]
    s_expect = s_expect*quasi_dists_neu[state]
    print(100*quasi_dists_neu[state], "% ",pl_string, s_expect)
    expect += s_expect
print("Expectation value for card values", c_vals, ":", expect)
#################################################